<a href="https://colab.research.google.com/github/AdamAnderB/Bramlett_DSPN_S24/blob/main/Copy_of_mixed_effects_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 10: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models.

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again.

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file.

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**.

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [ ]:
# WRITE YOUR CODE HERE
url <- "https://raw.githubusercontent.com/CoAxLab/DataSciencePsychNeuro/master/Homework%20datasets/lexDat/Items.csv"
file <- "Items.csv"
download.file(url, file)
items<-read.csv("Items.csv")

url <- "https://raw.githubusercontent.com/CoAxLab/DataSciencePsychNeuro/master/Homework%20datasets/lexDat/LexicalData.csv"
file <- "LexicalData.csv"
download.file(url, file)
lexical<-read.csv("LexicalData.csv")
library(tidyverse)



In [ ]:
lexical_items <- left_join(lexical, items, by = c("D_Word" = "Word"))
lexical_items<-lexical_items%>%
  drop_na()%>%
  mutate(D_RT= str_replace_all(D_RT, ",", ""))%>%
  mutate(D_RT=as.numeric(D_RT))
head(lexical_items)

,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore,Occurrences,Length,Freq_HAL,Log_Freq_HAL
,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<dbl>
1,157,1,1,710,browse,false,-0.437,2,6,"7,016",8.856
2,67,1,1,1094,refrigerant,false,0.825,3,11,104,4.644
3,120,1,1,587,gaining,false,-0.645,4,7,"4,039",8.304
4,21,1,1,984,cheerless,false,0.025,4,9,14,2.639
5,236,1,1,577,pattered,false,-0.763,4,8,4,1.386
6,236,2,1,715,conjures,false,-0.364,4,8,194,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output.

In [ ]:
# WRITE YOUR CODE HERE
simp_m<-lm(D_RT~Log_Freq_HAL*Length,data=lexical_items)
summary(simp_m)


Call:
lm(formula = D_RT ~ Log_Freq_HAL * Length, data = lexical_items)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Now, install `lme4` using `install.packages()` and then load the library.

In [ ]:
# WRITE YOUR CODE HERE
#install.packages("lme4")
#install.packages("lmerTest")
library(lme4)
library(lmerTest)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘nloptr’, ‘Rcpp’, ‘RcppEigen’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘numDeriv’


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output.

In [ ]:
# WRITE YOUR CODE HERE
mem_m<-lmer(D_RT~Log_Freq_HAL*Length+(1|Sub_ID),data=lexical_items)
summary(mem_m)


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: D_RT ~ Log_Freq_HAL * Length + (1 | Sub_ID)
   Data: lexical_items

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                      Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)           616.8445    17.1522  1051.7517  35.963  < 2e-16 ***
Log_Freq_HAL           -7.4374     1.5830 62314.1248  -4.698 2.63e-06 ***
Length                 47.7477     1.3162 62313.3658  36.277  < 2e-16 ***
Log_Freq_HAL:Length    -2.8778     0.1888 62313.3660 -15.239  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Len

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why?

> *Write your response here*

listed here:
Log_Freq_HAL -3.061 --- -4.698
Length  29.175 ---   36.277
Log_Freq_HAL -12.528  ---  -15.239.

the mixed effect model has higher t values (aka further from 0).  That is the higher t-values indicate a greater distance from the null hypothesis value. the Mixed effect model finds a strong effect. This is happening because the randomness is being accounted for in the random differences in performance between participants individually.

Use the Aikeke Information Criterion (AIC) to compare these two models.

1.   List item
2.   List item

Which one is better?

In [ ]:
# WRITE YOUR CODE HERE
AIC(simp_m)
AIC(mem_m)


[1] 914436.4

[1] 888247.6

> *Write your response here*
> The mixed effect model is better because although it is more complex it has a lower AIC(which builds in a penalty for complexity). The lower AIC indicates that how well models explain for the variance in the data set considering complexity of the model with a preference for simpler models.

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set?

> *Write your response here*
> The trial number could also be a random effect. We would expect that as trial number gets larger some variation may occur do to either getting better at the task or because of exhaustion. If we use it as a random effect, our main effects are more interpretabile because we don't need to consider the change over the task as much.

**DUE:** 5pm EST, March 18, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*